# The Dice Game
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70%, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can play a simple dice game. Here are the basic rules:
* You start with 0 points
* Roll three fair six-sided dice
* Now choose one of the following:
 * Stick, accept the values shown. If two or more dice show the same values, then all of them are flipped upside down: 1 becomes 6, 2 becomes 5, 3 becomes 4, and vice versa. The total is then added to your points and this is your final score.
 * OR reroll the dice. You may choose to hold any combination of the dice on the current value shown. Rerolling costs you 1 point – so during the game and perhaps even at the end your score may be negative. You then make this same choice again.

The best possible score for this game is 18 and is achieved by rolling three 1s on the first roll.

The reroll penalty prevents you from rolling forever to get this score. If the value of the current dice is greater than the expected value of rerolling them (accounting for the penalty), then you should stick.

The optimal decision is independent of your current score. It does not matter whether it is your first roll with a current score of 0, or your twentieth roll with a current score of -19 (in which case a positive end score is impossible), in either of these cases if you roll three 6s (which, if you stick, will only add 3 points) then you still expect to get a *better* end score by rerolling and taking the penalty. Almost any other roll will beat it, so it's still the right choice to maximise your score.

It is pretty obvious that you should stick on three 1s, and reroll on three 6s. Should you hold any of the 6s when you reroll? What about other values? What should you do if the dice come up 3, 4, 5?

We do not know what numbers will come up when we roll, but we do know exactly what the probability of any given roll is. This is the point of the probabilistic reasoning section of the unit; if we can model the true probabilities then we can mathematically calculate the optimal policy. Not all real world situations use dice, but these techniques work well even if we can only estimate the true probabilities.

### Play The Game
You can play the game in the following cell. Change the `SKIP_GAME` constant to `False` to enable this cell. 
<br> *Make sure to change it back to `True` before submitting.*

In [1]:
SKIP_GAME = True
if not SKIP_GAME:
    %run dice_game.py

### Submission Requirements
The code supports playing this game with many possible modifications – you can change the number of dice, the values on the dice, or even make biased (weighted) dice that are more likely to roll certain values. More on this later.

For this assignment you will need to submit:
1. The agent implementations which answer the **4 assignments** – this notebook
 * Your code will be subject to automated testing, from which grades will be assigned depending on how well your agent plays the game (potentially with modifications)
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests and skeleton code included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section
 
### Choice of Algorithm
You can take any approach you like to write your agent. We have provided two examples for you, though these agents do not perform very well, and would not pass the assignment, they should help you structure your code.

We have covered *value iteration* in the unit, and this technique will work well here to produce a strategy for the game, which your agent can then follow. It is up to you to work out the parameters that will be suit the value iteration algorithm to maximise your score.

However, there are many other possible options. Simply calculating the expected value of a single roll will produce a much stronger strategy than playing randomly. You could also look up various other approaches that can be applied to Markov decision processes, such as policy iteration.

To get a high grade on this assignment will require a particularly efficient implementation value iteration with intelligent choice of parameters, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

Note that you can write your agent to support modified versions of the game, which is also demonstrated below. For submissions that support this, the tests will include modified versions of the game, some where the game is *more obvious*, and some where it is *less obvious* than the default rules. Getting high marks requires supporting this feature. For those who are struggling to write a good agent, supporting this feature will result in additional tests which are more forgiving, so this might be an attractive option.

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Dice Game Class
A class called `DiceGame` is provided within `dice_game.py` for you to use in your solution. Here is a demonstration of its features.

When you create a DiceGame object, by default you will get the rules as stated above. 
```python
game = DiceGame()
```
Creates a game with 3 normal 6-sided dice. 

You may wish to modify the game mechanics, and you can do this using the other constructor arguments, for example:
```python
game = DiceGame(dice=4, sides=3, values=[1, 2, 6], bias=[0.1, 0.1, 0.8], penalty=2)
```
will create a game where you roll 4 dice, each with 3 sides, labelled 1, 2, and 6, where each die is far more likely to roll a 6 than they are to roll a 1 or a 2, and furthermore the penalty for rerolling is now 2 points instead of 1. *Note: this does not necessarily result in an interesting game.*

In games with unusual values or sides (3-sided dice are unusual without trying to turn them upside down), when there are duplicates, `value[i]` becomes `value[-i]`. With odd-sided dice, the middle value will flip onto itself.

Once created, the `DiceGame` object can be run in two different modes, *simulation* and *analysis*. It is likely that you will mostly use [*analysis* mode](#Analysis-Mode) to derive your agent's behaviour, but either way some understanding of simulation mode might be useful.

### Simulation Mode
The object provides the methods required to simulate playing the game. This might be useful if you want to test your agent, or you just want to try playing the game yourself, as we did in the cell earlier. The current dice values are found by calling `get_dice_state()`, they will always be listed in ascending order. 

In [2]:
from dice_game import DiceGame
import numpy as np

# setting a seed for the random number generator gives repeatable results, making testing easier!
np.random.seed(111)

game = DiceGame()
game.get_dice_state()

(2, 3, 4)

To roll the dice, you call the `roll` method which takes one parameter: a tuple representing which dice you want to hold, numbered from zero. We rolled (2, 3, 4). Suppose we want to hold the 2, we would pass the tuple `(0,)` into the `roll` method (note we need to include the comma so that Python knows this is a tuple).

The `roll` method returns a tuple containing: the reward for this action, the new state, and whether the game is over.

In [3]:
reward, new_state, game_over = game.roll((0,))
print(reward)
print(new_state)
print(game_over)

-1
(2, 2, 5)
False


Now suppose we are happy and wish to stick to get our final score. We can call the `roll` method and supply a tuple containing all three dice.

In [4]:
reward, new_state, game_over = game.roll((0, 1, 2))
print(reward)
print(new_state)
print(game_over)

15
(5, 5, 5)
True


Remember that the return value is just the reward for the action, in this case 15. To get our final score we can inspect `game.score`. We rerolled once, so expect to get a score of 14.

In [5]:
print(game.score)

14


And to play again we can call `game.reset()` which returns the new starting dice.

In [6]:
game.reset()

(1, 1, 3)

### Analysis Mode
In analysis mode, you are not playing the game, but asking the object for *all possible outcomes of certain actions*.

First of all, it is useful to know that all the possible states and all the possible actions are stored inside the object. Try changing the game mechanics on the first line (e.g. add `dice=2` to the constructor) and see how the other information is updated.

In [7]:
game = DiceGame()
print(f"The first 5 of {len(game.states)} possible dice rolls are: {game.states[0:5]}")
print(f"The possible actions on any given turn are: {game.actions}")

The first 5 of 56 possible dice rolls are: [(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 1, 4), (1, 1, 5)]
The possible actions on any given turn are: [(), (0,), (1,), (2,), (0, 1), (0, 2), (1, 2), (0, 1, 2)]


Finally, the most important method is `get_next_states(action, dice_state)`. This allows you to get all the possible resulting states for any given state and action.

Earlier we had the roll of `(2, 3, 4)` and decided to hold the 2. The game can calculate all possible outcomes for us, and crucially will also give us the probability of each state occurring.

In [8]:
game = DiceGame()
states, game_over, reward, probabilities = game.get_next_states((0,), (2, 3, 4))
for state, probability in zip(states, probabilities):
    print(f"Would get roll of {state} with probability {probability}")

Would get roll of (1, 1, 2) with probability 0.02777777777777778
Would get roll of (1, 2, 2) with probability 0.055555555555555566
Would get roll of (1, 2, 3) with probability 0.055555555555555566
Would get roll of (1, 2, 4) with probability 0.055555555555555566
Would get roll of (1, 2, 5) with probability 0.055555555555555566
Would get roll of (1, 2, 6) with probability 0.05555555555555559
Would get roll of (2, 2, 2) with probability 0.02777777777777778
Would get roll of (2, 2, 3) with probability 0.055555555555555566
Would get roll of (2, 2, 4) with probability 0.055555555555555566
Would get roll of (2, 2, 5) with probability 0.055555555555555566
Would get roll of (2, 2, 6) with probability 0.05555555555555559
Would get roll of (2, 3, 3) with probability 0.02777777777777778
Would get roll of (2, 3, 4) with probability 0.055555555555555566
Would get roll of (2, 3, 5) with probability 0.055555555555555566
Would get roll of (2, 3, 6) with probability 0.05555555555555559
Would get roll o

The method also works consistently when all dice are held, reporting that this action would cause the game to be over and giving the reward. Note that the list of states returned contains the value `None`. This is to denote that the game has entered a terminal state – no further actions would be allowed. The game does not return the final dice here, because that is another valid state (from which there would still be actions available). Also, the `reward` value is not the same as the final `score` of any given game, because it does not include any possible previous penalties for rerolling.

In [9]:
states, game_over, reward, probabilities = game.get_next_states((0, 1, 2), (2, 2, 5))
print(states)
print(game_over)
print(reward)

[None]
True
15


## Creating Agents to Play the Game

Below this cell is where you will start writing code yourself. We will take you through multiple steps, each of which will produce a better performing agent. You can submit the strongest agent you have.

Let's start with some example agents, so you can see the format we will use. In the cell below are two agents which do not play particularly well. One always holds immediately, the other will keep re-rolling all dice until they get the best possible dice (`(1, 1, 1)` or `(1, 1, 6)`), ignoring the massive penalty this will incur from re-rolling. Neither of them is considering the probabilities involved in the game.

There is also a function which will run the game with an instance of a given agent. When you run the cell, it will simulate a game with each agent.

In [10]:
from abc import ABC, abstractmethod


class DiceGameAgent(ABC):
    def __init__(self, game):
        self.game = game
    
    @abstractmethod
    def play(self, state):
        pass


class AlwaysHoldAgent(DiceGameAgent):
    def play(self, state):
        return (0, 1, 2)


class PerfectionistAgent(DiceGameAgent):
    def play(self, state):
        if state == (1, 1, 1) or state == (1, 1, 6):
            return (0, 1, 2)
        else:
            return ()
        
        
def play_game_with_agent(agent, game, verbose=False):
    state = game.reset()
    
    if(verbose): print(f"Testing agent: \n\t{type(agent).__name__}")
    if(verbose): print(f"Starting dice: \n\t{state}\n")
    
    game_over = False
    actions = 0
    while not game_over:
        action = agent.play(state)
        actions += 1
        
        if(verbose): print(f"Action {actions}: \t{action}")
        _, state, game_over = game.roll(action)
        if(verbose and not game_over): print(f"Dice: \t\t{state}")

    if(verbose): print(f"\nFinal dice: {state}, score: {game.score}")
        
    return game.score


if __name__ == "__main__":
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    np.random.seed(1)
    
    game = DiceGame()
    
    agent1 = AlwaysHoldAgent(game)
    play_game_with_agent(agent1, game, verbose=True)
    
    print("\n")
    
    agent2 = PerfectionistAgent(game)
    play_game_with_agent(agent2, game, verbose=True)

Testing agent: 
	AlwaysHoldAgent
Starting dice: 
	(1, 1, 2)

Action 1: 	(0, 1, 2)

Final dice: (2, 6, 6), score: 14


Testing agent: 
	PerfectionistAgent
Starting dice: 
	(2, 3, 3)

Action 1: 	()
Dice: 		(3, 4, 5)
Action 2: 	()
Dice: 		(1, 2, 6)
Action 3: 	()
Dice: 		(3, 4, 5)
Action 4: 	()
Dice: 		(1, 2, 5)
Action 5: 	()
Dice: 		(2, 5, 6)
Action 6: 	()
Dice: 		(1, 6, 6)
Action 7: 	()
Dice: 		(1, 2, 6)
Action 8: 	()
Dice: 		(1, 3, 6)
Action 9: 	()
Dice: 		(2, 4, 5)
Action 10: 	()
Dice: 		(1, 5, 6)
Action 11: 	()
Dice: 		(5, 5, 6)
Action 12: 	()
Dice: 		(1, 2, 5)
Action 13: 	()
Dice: 		(2, 3, 6)
Action 14: 	()
Dice: 		(1, 1, 2)
Action 15: 	()
Dice: 		(2, 2, 5)
Action 16: 	()
Dice: 		(1, 3, 4)
Action 17: 	()
Dice: 		(1, 4, 5)
Action 18: 	()
Dice: 		(1, 3, 5)
Action 19: 	()
Dice: 		(1, 3, 4)
Action 20: 	()
Dice: 		(4, 4, 6)
Action 21: 	()
Dice: 		(1, 4, 6)
Action 22: 	()
Dice: 		(1, 3, 5)
Action 23: 	()
Dice: 		(1, 3, 6)
Action 24: 	()
Dice: 		(5, 5, 6)
Action 25: 	()
Dice: 		(3, 4, 5)
Ac

# Step 1 - Hybrid Agent

Up to this point, we have covered how the dice game works and provided some examples of simple agents that can play the game - specifically an agent that always holds regardless of the value on the dice, the `AlwaysHoldAgent`, and an agent that will continue to re-roll the dice until they get the perfect score, called the `PerfectionistAgent`.

Obviously, as the game progresses the reward available reduces as taking more re-rolls will eventually result in so many penalties that they will always outweigh any positive score obtained. So although the perfectionist may get lucky and get a high score early in the game if the agent takes too many goes then its score will be worse than had it just held onto what it already had. As such a hybrid strategy of the `AlwaysHoldAgent` and the `PerfectionistAgent` should on average outperform them individually given enough games.

Therefore your first task will be to create a hybrid agent of the `AlwaysHold` and `Perfectionist` agents. One thing you will need to investigate is when it is optimal to switch between the two strategies. When is better to chase perfection, and when is it best to just cut your losses and hold what you have?

We have written a function in the dice_game.py file to help you do this: get_dice_score(), which can be used to calculate the current score on the dice, not including any penalties for rerolling.

## Aims:

- Complete the function provided creating a hybrid of the `AlwaysHold` and `Perfectionist` agents
- Optimise the agent so that it outperforms both the `AlwaysHold` and `Perfectionist` agents given enough games

In [11]:
class HybridAgent(DiceGameAgent):
    def play(self, state):
        current_score = self.game.get_dice_score()
        
        # Your Code Here
        pass

# Step 2 - Manual Agent

So as you may have discovered even the best hyrid agent doesn't perform that well. You may have also found that you were able to obtain a better score playing the game yourself using your intuition.

So for the next assignment, your job will be to implement an agent that can outperform the `Hybrid`, `Perfectionist` and the `AlwaysHold` agents. Enter your code inside the play function below. You can use the init function to set properties if you would like, as follows:

```
def __init__(self, game):
    super().__init__(game)
    self.some_property = 10
```

Then it can be used inside the play function like so:

```
def play(self, state):
    a = self.some_property
```

## Aims:
- Implement an agent that plays how you might play.
- This agent should be able to outperform the three agents you have seen so far - the `Hybrid`, `Perfectionist` and the `AlwaysHold` agents.


**HINT** - This doesn't need to play perfectly, only a small number of modifications will be required to improve on the previous agents. Try not to overthink it, it is only expected that you find small improvements over the Hybrid Agent.

In [12]:
class ManualAgent(DiceGameAgent):
    
    def __init__(self, game):
        super().__init__(game)
    
    def play(self, state):
        # Your Code Here.
        pass


# Value Iteration

Up to this point, we have focused on implementing manually created strategies such as always holding regardless of the dice values. However, the approaches taken up to this point are sub-optimal and although it may be possible to hand-craft the optimal policy, it would be very difficult, and even impossible if the problem was more complex. As such there are ways of autonomously learning the optimal policy, some of which we have covered on the course.

Specifically, for this coursework, we will be focusing on **value iteration**, which is a method for learning the expected future discounted reward for every state in a game. Using this expected future reward it is possible to derive the optimal policy to take by selecting the actions that move the agent to the state with the highest expected future reward.

![Value Iteration Algorithm](ValueIterationAlgorithm.png)

The value iteration algorithm is provided here for your convinence and it is the algorithm that you will have to implment over the following assignments.

# Step 3 - One Step Value Iteration

For assignment 3 the goal will be to implement one step value iteration. The only difference between this and full value iteration is that values are only updated once, rather than continually updating them until they are within some threshold. We will be extending the work here for the final assignment to implement the full algorithm, however, for this assignment, we will only implement the modified algorithm provided here:

![Value Iteration Algorithm](OneStepValueIterationAlgorithm.png)

For your agent you should use the following parameter values (these are already set in the init function so please don't change them):

- Gamma: 1.0


## Aims
- Complete the `init` and `perform_single_value_iteration` functions to perform a single step of value iteration.
- Complete the `play` function that takes the optimal action for a given state according to the values learned by the previous functions.

NOTE. One step value iteration should call the `perform_single_value_iteration` twice, as the first call will not actually step the agent, it will only initialize the values, so the second call will perform the first step.

In [13]:
class OneStepValueIterationAgent(DiceGameAgent):
    def __init__(self, game):
        """
        If your code does any pre-processing on the game, you can do it here.
        
        You can always access the game with self.game
        """
        super().__init__(game)
        self.gamma = 1.0
        
        # Your Code Here
        
        _, self.values = self.perform_single_value_iteration()
        _, self.values = self.perform_single_value_iteration()
    
    def perform_single_value_iteration(self):
        # Your Code Here
        
        return delta, new_values
    
    def play(self, state):
        """
        given a state, return the chosen action for this state
        at minimum you must support the basic rules: three six-sided fair dice
        
        if you want to support more rules, use the values inside self.game, e.g.
            the input state will be one of self.game.states
            you must return one of self.game.actions
        
        read the code in dicegame.py to learn more
        """
        # Your Code Here
        pass

# Step 4 - Full Value Iteration

Having completed a single step value iteration you can finsih the full algorithm shown here:

![Value Iteration Algorithm](ValueIterationAlgorithm.png)

There are 4 functions to complete as a part of this exercise, you have already done 3 of them so you are more than welcome to copy their contents and re-use them again here. In addition, you must complete the `value_iteration` that will call the `perform_single_value_iteration` function until the threshold is met. This will also require some small changes to the `init` function. Finally, you must investigate what are the optimal parameters for the algorithm in order to get the best results. **This part is very important!**

## Aims

- Correctly implement full value iteration
- Optimise the parameters (gamma, theta) to get the best agent possible



In [14]:
import time

class ValueIterationAgent(DiceGameAgent):
    def __init__(self, game):
        """
        If your code does any pre-processing on the game, you can do it here
        
        You can always access the game with self.game
        """
        super().__init__(game)
        
        # These are default values, you can play around with these to improve your agents performance.
        self.theta = 0.01
        self.gamma = 0.9
        
        # YOUR CODE HERE
        raise NotImplementedError()
        
    
    def perform_single_value_iteration(self):
        # We advice you use the function from the previous assignment. You do not have to if you don't want to but it should save you alot of time.
        # YOUR CODE HERE
        raise NotImplementedError()
        
        return delta, new_values
    
    def value_iteration(self, theta=0.001):
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def play(self, state):
        """
        given a state, return the chosen action for this state
        at minimum you must support the basic rules: three six-sided fair dice
        
        if you want to support more rules, use the values inside self.game, e.g.
            the input state will be one of self.game.states
            you must return one of self.game.actions
        
        read the code in dicegame.py to learn more
        """
        # YOUR CODE HERE
        raise NotImplementedError()

# Submission Point

Below, you should choose your best performing agent to submit for this coursework. You can simply copy and paste the code you want to use from a previous task. The following is a guide to the mark you should expect for a correct implementation of each step:

- Hybrid Agent: 40-50 marks.
- Manual Agent: 40-60 marks.
- One Step Value Iteration: 60-70 marks
- Non-optimal value Iteration: 70-80 marks
- Optimal Value Iteration + Extended Rules Compatability: 80+ marks

Please note that these are only rough guidelines: your actual mark will depend on the performance of your agent.

In [15]:
def calculate_score(dice, state):                  # this function returns the score of the next state
    if len(state) == len(set(state)):
        return sum(state)      # returns the score of the next value
    else:
        i_list = []
        lst = list(state)
        for a in range(0,len(lst)):
            for b in range(a+1, len(lst)):
                if lst[a] == lst[b]:
                    i_list.append(a)
                    i_list.append(b)
        i_list = list(set(i_list))
        
        for index in i_list:
                swap = -dice.game._values[list(dice.game._values).index(lst[index])]
                lst[index] = dice.game._values[swap]
        return sum(lst)       # returns the score of the next value


class ValueIterationAgent(DiceGameAgent):

    def __init__(self, game):
        super().__init__(game)
        self.theta = 0.1         # initializing the value of theta 
        self.gamma = 0.96           # initializing the value of gamma 
        # the values of theta and gamma were found through hit and trial
        self.v = self.value_iteration()

    def value_iteration(self):
        game  = DiceGame()

        V_opt = {}              # stores the V_opt values
        m_action = 0            # stores the max action
        policy = {}              # stores the policy for each state
        V_sum = 0                # stores the sum of V_opt
        delta = 0              #  V_opt(s)-V_opt(s')
        state_val = {}          # value of the state
        delta_max = self.theta+1  # used as a buffer marker

        for s in game.states:     
            policy[s] = ()      
            V_opt[s] = 0        # assigning the values of V_opt to 0 for 1st iteration 

        while (delta_max >= self.theta):    
            delta_max = 0

            for state in game.states:      # gets all the states 
                state_val = V_opt[state]        # state_val takes a temporary value of V_opt[]
                m_action = 0

                for action in game.actions:
                    V_sum = 0                      
                    states, game_over, reward, probabilities = game.get_next_states(action, state)  
                    # these line was copied from the dicegame file, which essentially rus through each action and gets its values for each individual state, reward, game_over status and transition probabilities

                    for s, p in zip(states, probabilities): # cycles through each states probablity 

                        if game_over == False:

                            V_sum = V_sum + p*(calculate_score(self, s) - calculate_score(self, state) + reward+ self.gamma * V_opt[s])   # gets the sum of all V_opts for all the states of each action
                            #instead of using a fixed reward value we are subtracting the probablity of the next value from the previous value
                        else:
                            V_sum = reward - 10.5 - 3

                    if V_sum > m_action:       # this checks is the new value of sum is greater than the previous value of m_action
                        m_action = V_sum    # if so m_action gets updated
                        policy[state] = action

                delta = abs(state_val - V_opt[state])  # calculates the delta between the 2 V_opts
                if(delta > delta_max):
                    delta_max = delta      # delta max is the final difference that is taken use of for in the while statement

                V_opt[state] = m_action
        return policy       # returns a policy

    def play(self, state):
        return self.v[state]  

In [16]:
if __name__ == "__main__":
    # random seed makes the results deterministic
    # change the number to see different results
    # or delete the line to make it change each time it is run
    np.random.seed(1)
    
    game = DiceGame()
    
    agent1 = ValueIterationAgent(game)
    play_game_with_agent(agent1, game, verbose=True)

Testing agent: 
	ValueIterationAgent
Starting dice: 
	(2, 3, 3)

Action 1: 	(1, 2)
Dice: 		(3, 3, 4)
Action 2: 	()
Dice: 		(2, 3, 5)
Action 3: 	(0,)
Dice: 		(1, 2, 6)
Action 4: 	(0,)
Dice: 		(1, 3, 5)
Action 5: 	(0,)
Dice: 		(1, 1, 4)
Action 6: 	(0, 1, 2)

Final dice: (4, 6, 6), score: 11


# Opt-In For Extended Rules

If you wish your code to be tested with rules other than the defaults, set the constant `TEST_EXTENDED_RULES` to `True` on the next line. Another test will be performed to check your code still works. If you get an error, you are likely not supporting the extended rules properly.

Refer to [this section](#Choice-of-Algorithm) to understand more about how extended rules factor into your possible grade.

**Note:** you need to have `SKIP_TESTS` set to `False` in the cell above (and run it!) to enable the tests below. The value of `TEST_EXTENDED_RULES` *will* be used in the grading tests.

In [17]:
import time

SKIP_TESTS = False
TEST_EXTENDED_RULES = True

def extended_tests():
    import time 
    total_score = 0
    total_time = 0
    n = 10

    print("Testing extended rules – two three-sided dice.")
    print()

    game = DiceGame(dice=2, sides=3)

        
    start_time = time.process_time()
    test_agent = MyAgent(game)
    total_time += time.process_time() - start_time

    for i in range(n):
        start_time = time.process_time()
        score = play_game_with_agent(test_agent, game)
        total_time += time.process_time() - start_time

        print(f"Game {i} score: {score}")
        total_score += score

    print()
    print(f"Average score: {total_score/n}")
    print(f"Average time: {total_time/n:.5f} seconds")

    

if not SKIP_TESTS and TEST_EXTENDED_RULES:
    extended_tests()

Testing extended rules – two three-sided dice.



NameError: name 'MyAgent' is not defined

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [ ]:
import sys
import pathlib

fail = False;

if not SKIP_GAME:
    fail = True;
    print("You must set the SKIP_GAME constant to True in the earlier cell.")

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the earlier cell.")


p3 = pathlib.Path('./dicegame.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named dicegame.ipynb")
    
if "MyAgent" not in dir():
    fail = True;
    print("You must include a class called MyAgent as defined above.")
else:    
    game = DiceGame()
    agent = MyAgent(game)
    action = agent.play((1, 1, 1))

    if action not in game.actions:
        print("Warning:")
        print("Your agent does not seem to produce a valid action with the default rules.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
        
    if TEST_EXTENDED_RULES:
        print("Info: TEST_EXTENDED_RULES is ON")
        game = DiceGame(dice=2, sides=8)
        agent = MyAgent(game)
        try:
            action = agent.play((7, 8))
        except:
            action = None

        if action not in game.actions:
            fail = True
            print("Your agent does not produce a valid action with the extended rules.")
            print("Turn off TEST_EXTENDED_RULES if you cannot fix this error.")
    else:
        print("Info: TEST_EXTENDED_RULES is OFF (extended rules will not be tested)")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print()
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

In [ ]:
# Auto-Marked cell, DO NOT DELTE, MODIFY OR DUPLICATE!
